In [3]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings 
warnings.filterwarnings('ignore')

In [4]:
def clean_games(games,team_name):
    games = games.copy()
    games_column = ["idx","G" ,"Date" ,"Start (ET)","Team_Name","BoxScore", "Venue","Opponent" ,"Result",
                    " Nan2","Tm" ,"Opp" ,"W_Streak" ,"L_Streak" ,"Streak" ,"Notes"]
    games.columns = games_column
    games["Team_Name"] = team_name
    
    indices_to_keep = (games['G']!= 'G') & (games['G'].str.isdigit())
    games = games[indices_to_keep].reset_index(drop=True)
    games['G'] = games.G.astype('int')
    games = games[games.G<=82] # get only regular season
    
    for col in ["W_Streak" ,"L_Streak" ,"Streak"]:
        stk = games[col].values # streak_values
        stk = np.delete(stk,-1)
        stk = np.insert(stk,0,np.nan)
        games[col] = stk
    use_col = ['idx', 'G', 'Date', 'Start (ET)', 'Team_Name', 'Venue','Opponent', 'Result', 'Tm', 'Opp',"W_Streak" ,"L_Streak", 'Streak']
    games=games[use_col]
    print("shape of games_df",games.shape)
    return games

In [12]:
def clean_lineups(line_ups):
    if line_ups.G.dtype =='int64':
        indices_to_keep = (line_ups['G']!= 'G')# & (coaches['coach_To'].str.isdigit())
    else:
        indices_to_keep = (line_ups['G']!= 'G') & (line_ups['G'].str.isdigit())
    
    line_ups = line_ups[indices_to_keep].reset_index(drop=True)
    line_ups =line_ups[['G','Starting Lineup']]
    print("shape of line_ups_df",line_ups.shape)
    return line_ups

In [5]:
def clean_gamelog(gamelog):
    gamelog = gamelog.copy()
    col1 = ["idx", 'Rk', 'G', 'Date', 'Venue', 'Opponent', 'W/L', 'Tm',
       'Opp']
    common_col = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',]
    team_col = ["Tm_"+col for col in common_col]
    opp_col = ["Opp_"+col for col in common_col]
    gamelog_columns = col1+team_col+['Nan1']+opp_col
    gamelog.columns = gamelog_columns
    indices_to_keep = (gamelog['G']!= 'G') & (gamelog['G'].str.isdigit())
    game_log = gamelog[indices_to_keep].reset_index(drop=True)
    game_log = game_log.drop(['idx', 'Rk', 'Date', 'Venue', 'Opponent', 'W/L', 'Tm', 'Opp','Nan1'],axis=1)
    game_log['G'] = game_log.G.astype('int')
    game_log = game_log[game_log.G<=82]
    print("shape of gamelog_df",game_log.shape)
    return game_log

In [6]:
def clean_game_advance_stats(game_advance_stats):
    game_advance_stats= game_advance_stats.copy()
    
    col1=['idx', 'Rk', 'G', 'Date', 'Venue', 'Opponent', 'W/L', 'Tm','Opp']
    advanced = ["Adv_Tm_"+col for col in 
            ['ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%','AST%', 'STL%', 'BLK%']]
    Nan1 = ["Nan1"]
    offensive4F = ["Adv_Tm_O_4F_"+col for col in 
    ['eFG%', 'TOV%','ORB%', 'FT/FGA']]
    Nan2 = ["Nan2"]
    deffensive4F = ["Adv_Tm_D_4F_"+col for col in 
    ['eFG%', 'TOV%','DRB%', 'FT/FGA']]
    
    advanced_columns = col1+advanced+['Nan1']+offensive4F+Nan2+deffensive4F
    game_advance_stats.columns = advanced_columns
    
    indices_to_keep = (game_advance_stats['G'] != 'G') & (game_advance_stats['G'].str.isdigit()) #& (game_advance_stats['G'].str.isdigit())
    game_advance_stats = game_advance_stats[indices_to_keep].reset_index(drop=True)
    
    game_advance_stats = game_advance_stats.drop(['idx', 'Rk', 'Date', 'Venue', 'Opponent', 'W/L', 'Tm', 'Opp','Nan1','Nan2'],axis=1)
    game_advance_stats['G'] = game_advance_stats.G.astype('int')
    game_advance_stats = game_advance_stats[game_advance_stats.G<=82]
    print("shape of game_advance_stats_df",game_advance_stats.shape)
    return game_advance_stats#game_advance_stats[indices_to_keep].reset_index(drop=True)#game_advance_stats

In [7]:
def Get_H2H(H2H,games):
    H2H = H2H.copy()
    games = games.copy()
    H2H_df=H2H[H2H.Franchise.isin(games.Opponent.unique())].reset_index(drop=True)
    #display(H2H_df.head())
    val_count = H2H_df.Franchise.value_counts()
    if len(val_count[H2H_df.Franchise.value_counts()>1])>0:
        max_game = H2H_df[H2H_df.Franchise=='Denver Nuggets']['G'].max()
        H2H_df = H2H_df.drop(H2H_df[(H2H_df.Franchise=='Denver Nuggets')&(H2H_df.G!=max_game)].index)
    H2H_df.drop(['Unnamed: 0','Rk'],axis=1,inplace=True)
        #H2H_df = H2H_df.drop(['Unnamed: 0','Rk'],axis=1)
    H2H_df.columns = ['Franchise']+ ["H2H_"+col for col in 
            ['G', 'W', 'L', 'W/L%', 'PS', 'PA','PS/G', 'PA/G']]
    return H2H_df

In [8]:
def clean_prev_season_and_coach(prev_season,coaches,team_name):
    prev_season = prev_season.copy()
    coaches = coaches.copy()
    s20_21 = prev_season[prev_season.Season=='2020-21']
    s20_21 = s20_21[['Team','W/L%', 'Finish', 'SRS', 'Pace', 'Rel Pace', 'ORtg', 'Rel ORtg', 'DRtg','Rel DRtg','Playoffs', 'Top WS']]
    s_columns = ['Team'] + ["prev_"+col  for col in
                        ['W/L%', 'Finish', 'SRS', 'Pace', 'Rel_Pace', 'ORtg', 'Rel_ORtg', 'DRtg','Rel_DRtg','Playoffs', 'Top_WS']]
    s20_21.columns = s_columns 
    s20_21.loc[:,"Team"] = s20_21["Team"].str.replace("[^A-Za-z]+", "", regex=True)
    #display(s20_21)
    coaches.columns =['idx', 'Rk', 'Coach']+  [ 'coach_'+col for col in
        ['From', 'To', 'Yrs', 'G', 'W', 'L', 'W/L%',
       'W > .500', 'Plyfs', 'G', 'W', 'L', 'W/L%', 'Conf', 'Champ']]
    if coaches.coach_To.dtype =='int64':
        indices_to_keep = (coaches['coach_To']!= 'coach_To')# & (coaches['coach_To'].str.isdigit())
    else:
        indices_to_keep = (coaches['coach_To']!= 'coach_To') & (coaches['coach_To'].str.isdigit())
    coaches = coaches[indices_to_keep].reset_index(drop=True)
    present_coach = coaches[coaches.coach_To == coaches.coach_To.max()]
    present_team = s20_21["Team"].values[0]
    present_coach['Team']=present_team
    present_coach['Team_abv']= team_name
    #display(present_coach)
    coach_prev_season = pd.merge(left=present_coach,right=s20_21,on='Team')
    coach_prev_season = coach_prev_season.drop(['idx', 'Rk'],axis=1)
    return coach_prev_season

In [9]:
games = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\games.csv")#("C:/Users/user/Solo Dolo/NBA_Analysis/Game/DET/games.csv")

In [14]:
a=clean_games(games,'ATL')

shape of games_df (82, 13)


In [68]:
games.head(2)

,Unnamed: 0,G,Date,Start (ET),Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes
0,0,1,"Thu, Oct 21, 2021",7:30p,NaN,Box Score,NaN,Dallas Mavericks,W,NaN,113,87,1,0,W 1,NaN
1,1,2,"Sat, Oct 23, 2021",6:00p,NaN,Box Score,@,Cleveland Cavaliers,L,NaN,95,101,1,1,L 1,NaN


In [10]:
line_ups = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\games_starting_lineup.csv")

In [13]:
al = clean_lineups(line_ups)

shape of line_ups_df (82, 2)


In [15]:
a.head()

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2


In [16]:
al.head()

,G,Starting Lineup
0,1,B. Bogdanović · C. Capela · J. Collins · D. Hu...
1,2,B. Bogdanović · C. Capela · J. Collins · D. Hu...
2,3,B. Bogdanović · C. Capela · J. Collins · K. Hu...
3,4,B. Bogdanović · C. Capela · J. Collins · D. Hu...
4,5,B. Bogdanović · C. Capela · J. Collins · D. Hu...


In [17]:
a = pd.merge(left = a , right = al, how='left', on = 'G')

In [18]:
a.head()

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,B. Bogdanović · C. Capela · J. Collins · D. Hu...
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,B. Bogdanović · C. Capela · J. Collins · K. Hu...
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,B. Bogdanović · C. Capela · J. Collins · D. Hu...


In [7]:
gamelog = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\gamelog.csv")#("C:/Users/user/Solo Dolo/NBA_Analysis/Game/DET/games.csv")

In [11]:
b=clean_gamelog(gamelog)

shape of gamelog_df (82, 33)


In [12]:
b

,G,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF
0,1,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21
1,2,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16
2,3,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15
3,4,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19
4,5,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,36,86,.419,13,41,.317,37,49,.755,10,54,18,10,4,6,18,42,99,.424,17,37,.459,14,19,.737,10,52,17,5,3,13,29
78,79,43,94,.457,14,37,.378,8,12,.667,10,43,31,5,4,7,22,44,101,.436,9,35,.257,21,31,.677,20,60,20,6,5,7,14
79,80,43,92,.467,16,39,.410,16,19,.842,7,39,26,11,1,6,13,42,86,.488,10,35,.286,9,11,.818,6,44,26,4,4,14,17
80,81,42,88,.477,8,34,.235,17,19,.895,13,43,17,9,0,14,23,40,72,.556,11,26,.423,22,28,.786,4,33,20,9,1,17,17


In [52]:
ab = pd.merge(left = a , right = b, how='left', on = 'G')

In [53]:
ab

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,80,78,"Sat, Apr 2, 2022",7:30p,ATL,NaN,Brooklyn Nets,W,122,115,40,37,W 4,36,86,.419,13,41,.317,37,49,.755,10,54,18,10,4,6,18,42,99,.424,17,37,.459,14,19,.737,10,52,17,5,3,13,29
78,81,79,"Tue, Apr 5, 2022",7:30p,ATL,@,Toronto Raptors,L,108,118,41,37,W 5,43,94,.457,14,37,.378,8,12,.667,10,43,31,5,4,7,22,44,101,.436,9,35,.257,21,31,.677,20,60,20,6,5,7,14
79,82,80,"Wed, Apr 6, 2022",8:00p,ATL,NaN,Washington Wizards,W,118,103,41,38,L 1,43,92,.467,16,39,.410,16,19,.842,7,39,26,11,1,6,13,42,86,.488,10,35,.286,9,11,.818,6,44,26,4,4,14,17
80,84,81,"Fri, Apr 8, 2022",8:00p,ATL,@,Miami Heat,L,109,113,42,38,W 1,42,88,.477,8,34,.235,17,19,.895,13,43,17,9,0,14,23,40,72,.556,11,26,.423,22,28,.786,4,33,20,9,1,17,17


In [15]:
game_advance_stats = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\game_advance_stats.csv")

In [16]:
c=clean_game_advance_stats(game_advance_stats)

shape of game_advance_stats_df (82, 19)


In [17]:
c

,G,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA
0,1,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129
1,2,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225
2,3,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165
3,4,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086
4,5,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,116.4,109.7,104.8,.570,.477,.567,50.9,50.0,9.5,6.5,.494,5.3,19.2,.430,.510,10.8,81.5,.141
78,79,112.3,122.7,96.1,.128,.394,.544,41.7,72.1,5.2,6.1,.532,6.6,20.0,.085,.480,5.8,62.3,.208
79,80,121.4,106.0,97.2,.207,.424,.588,47.0,60.5,11.3,2.0,.554,5.6,15.6,.174,.547,13.4,84.2,.105
80,81,114.4,118.6,95.3,.216,.386,.566,56.6,40.5,9.4,0.0,.523,12.7,31.0,.193,.632,16.8,88.2,.306


In [54]:
abc = pd.merge(left = ab , right = c, how='left', on = 'G')

In [55]:
abc

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,80,78,"Sat, Apr 2, 2022",7:30p,ATL,NaN,Brooklyn Nets,W,122,115,40,37,W 4,36,86,.419,13,41,.317,37,49,.755,10,54,18,10,4,6,18,42,99,.424,17,37,.459,14,19,.737,10,52,17,5,3,13,29,116.4,109.7,104.8,.570,.477,.567,50.9,50.0,9.5,6.5,.494,5.3,19.2,.430,.510,10.8,81.5,.141
78,81,79,"Tue, Apr 5, 2022",7:30p,ATL,@,Toronto Raptors,L,108,118,41,37,W 5,43,94,.457,14,37,.378,8,12,.667,10,43,31,5,4,7,22,44,101,.436,9,35,.257,21,31,.677,20,60,20,6,5,7,14,112.3,122.7,96.1,.128,.394,.544,41.7,72.1,5.2,6.1,.532,6.6,20.0,.085,.480,5.8,62.3,.208
79,82,80,"Wed, Apr 6, 2022",8:00p,ATL,NaN,Washington Wizards,W,118,103,41,38,L 1,43,92,.467,16,39,.410,16,19,.842,7,39,26,11,1,6,13,42,86,.488,10,35,.286,9,11,.818,6,44,26,4,4,14,17,121.4,106.0,97.2,.207,.424,.588,47.0,60.5,11.3,2.0,.554,5.6,15.6,.174,.547,13.4,84.2,.105
80,84,81,"Fri, Apr 8, 2022",8:00p,ATL,@,Miami Heat,L,109,113,42,38,W 1,42,88,.477,8,34,.235,17,19,.895,13,43,17,9,0,14,23,40,72,.556,11,26,.423,22,28,.786,4,33,20,9,1,17,17,114.4,118.6,95.3,.216,.386,.566,56.6,40.5,9.4,0.0,.523,12.7,31.0,.193,.632,16.8,88.2,.306


In [28]:
H2H = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\H2H.csv")

In [29]:
d=Get_H2H(H2H,games)

In [30]:
d

,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G
0,Boston Celtics,386,147,239,0.381,39586,40936,102.6,106.1
1,Brooklyn Nets,188,103,85,0.548,19607,19056,104.3,101.4
2,Charlotte Hornets,126,67,59,0.532,12888,12741,102.3,101.1
3,Chicago Bulls,252,117,135,0.464,25556,25714,101.4,102.0
4,Cleveland Cavaliers,236,127,109,0.538,24192,24058,102.5,101.9
5,Dallas Mavericks,82,45,37,0.549,8398,8278,102.4,101.0
7,Denver Nuggets,95,51,44,0.537,10111,10116,106.4,106.5
8,Detroit Pistons,438,213,225,0.486,44340,44530,101.2,101.7
9,Golden State Warriors,297,147,150,0.495,31242,31384,105.2,105.7
10,Houston Rockets,160,79,81,0.494,17079,17151,106.7,107.2


In [58]:
abcd = pd.merge(left = abc , right = d, how='left', left_on = 'Opponent', right_on = 'Franchise')

In [59]:
abcd

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129,Dallas Mavericks,82,45,37,0.549,8398,8278,102.4,101.0
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225,Cleveland Cavaliers,236,127,109,0.538,24192,24058,102.5,101.9
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165,Detroit Pistons,438,213,225,0.486,44340,44530,101.2,101.7
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086,New Orleans Pelicans,41,21,20,0.512,3986,4019,97.2,98.0
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172,Washington Wizards,312,162,150,0.519,32954,32728,105.6,104.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,80,78,"Sat, Apr 2, 2022",7:30p,ATL,NaN,Brooklyn Nets,W,122,115,40,37,W 4,36,86,.419,13,41,.317,37,49,.755,10,54,18,10,4,6,18,42,99,.424,17,37,.459,14,19,.737,10,52,17,5,3,13,29,116.4,109.7,104.8,.570,.477,.567,50.9,50.0,9.5,6.5,.494,5.3,19.2,.430,.510,10.8,81.5,.141,Brooklyn Nets,188,103,85,0.548,19607,19056,104.3,101.4
78,81,79,"Tue, Apr 5, 2022",7:30p,ATL,@,Toronto Raptors,L,108,118,41,37,W 5,43,94,.457,14,37,.378,8,12,.667,10,43,31,5,4,7,22,44,101,.436,9,35,.257,21,31,.677,20,60,20,6,5,7,14,112.3,122.7,96.1,.128,.394,.544,41.7,72.1,5.2,6.1,.532,6.6,20.0,.085,.480,5.8,62.3,.208,Toronto Raptors,99,49,50,0.495,9906,9956,100.1,100.6
79,82,80,"Wed, Apr 6, 2022",8:00p,ATL,NaN,Washington Wizards,W,118,103,41,38,L 1,43,92,.467,16,39,.410,16,19,.842,7,39,26,11,1,6,13,42,86,.488,10,35,.286,9,11,.818,6,44,26,4,4,14,17,121.4,106.0,97.2,.207,.424,.588,47.0,60.5,11.3,2.0,.554,5.6,15.6,.174,.547,13.4,84.2,.105,Washington Wizards,312,162,150,0.519,32954,32728,105.6,104.9
80,84,81,"Fri, Apr 8, 2022",8:00p,ATL,@,Miami Heat,L,109,113,42,38,W 1,42,88,.477,8,34,.235,17,19,.895,13,43,17,9,0,14,23,40,72,.556,11,26,.423,22,28,.786,4,33,20,9,1,17,17,114.4,118.6,95.3,.216,.386,.566,56.6,40.5,9.4,0.0,.523,12.7,31.0,.193,.632,16.8,88.2,.306,Miami Heat,130,57,73,0.438,12544,12894,96.5,99.2


In [19]:
prev_season = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\prev_season.csv")

In [20]:
coaches = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\ATL\coaches.csv",skiprows=1)

In [49]:
coaches

,Unnamed: 0,Rk,Coach,From,To,Yrs,G,W,L,W/L%,W > .500,Plyfs,G.1,W.1,L.1,W/L%.1,Conf,Champ
0,0,1,Red Auerbach,1950,1950,1,57,28,29,.491,-0.5,1,3,1,2,.333,0,0
1,1,2,Hubie Brown,1977,1981,5,407,199,208,.489,-4.5,3,16,6,10,.375,0,0
2,2,3,Mike Budenholzer,2014,2018,5,410,213,197,.520,8.0,4,39,17,22,.436,0,0
3,3,4,Larry Drew,2011,2013,3,230,128,102,.557,13.0,3,24,10,14,.417,0,0
4,4,5,Cotton Fitzsimmons,1973,1976,4,320,140,180,.438,-20.0,1,6,2,4,.333,0,0
5,5,6,Mike Fratello,1981,1990,8,577,324,253,.562,35.5,5,40,18,22,.450,0,0
6,6,7,Harry Gallatin,1963,1965,3,193,111,82,.575,14.5,2,23,12,11,.522,0,0
7,7,8,Richie Guerin,1965,1972,8,618,327,291,.529,18.0,8,60,26,34,.433,0,0
8,8,9,Alex Hannum,1957,1958,2,103,56,47,.544,4.5,2,21,14,7,.667,0,1
9,9,10,Red Holzman,1954,1957,4,203,83,120,.409,-18.5,1,8,4,4,.500,0,0


In [21]:
e=clean_prev_season_and_coach(prev_season,coaches,'ATL')

<ipython-input-18-5e8c928dd9de>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  present_coach['Team']=present_team
<ipython-input-18-5e8c928dd9de>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  present_coach['Team_abv']= team_name


In [22]:
e

,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)


In [61]:
abcde = pd.merge(left = abcd , right = e, how='left', left_on = 'Team_Name', right_on = 'Team_abv')

In [62]:
abcde

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129,Dallas Mavericks,82,45,37,0.549,8398,8278,102.4,101.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225,Cleveland Cavaliers,236,127,109,0.538,24192,24058,102.5,101.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165,Detroit Pistons,438,213,225,0.486,44340,44530,101.2,101.7,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086,New Orleans Pelicans,41,21,20,0.512,3986,4019,97.2,98.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172,Washington Wizards,312,162,150,0.519,32954,32728,105.6,104.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,80,78,"Sat, Apr 2, 2022",7:30p,ATL,NaN,Brooklyn Nets,W,122,115,40,37,W 4,36,86,.419,13,41,.317,37,49,.755,10,54,18,10,4,6,18,42,99,.424,17,37,.459,14,19,.737,10,52,17,5,3,13,

In [67]:
abcde.columns

Index(['idx', 'G', 'Date', 'Start (ET)', 'Team_Name', 'Venue', 'Opponent',
       'Result', 'Tm', 'Opp',
       ...
       'prev_Finish', 'prev_SRS', 'prev_Pace', 'prev_Rel_Pace', 'prev_ORtg',
       'prev_Rel_ORtg', 'prev_DRtg', 'prev_Rel_DRtg', 'prev_Playoffs',
       'prev_Top_WS'],
      dtype='object', length=101)

In [19]:
def merge_df(team):
    #games = pd.read_csv("C:/Users/user/Solo Dolo/NBA_Analysis/Game\"+ATL"\games.csv")
    games = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//games.csv")
    a=clean_games(games,team)
    
    line_ups = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//games_starting_lineup.csv")
    al = clean_lineups(line_ups)
    a = pd.merge(left = a , right = al, how='left', on = 'G')
    
    gamelog = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//gamelog.csv")
    b=clean_gamelog(gamelog)
    
    ab = pd.merge(left = a , right = b, how='left', on = 'G')
    print("ab.shape : ",ab.shape)
    
    game_advance_stats = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//game_advance_stats.csv")
    c=clean_game_advance_stats(game_advance_stats)
    
    abc = pd.merge(left = ab , right = c, how='left', on = 'G')
    print("abc.shape : ",abc.shape)
    
    H2H = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//H2H.csv")
    d=Get_H2H(H2H,games)
    
    abcd = pd.merge(left = abc , right = d, how='left', left_on = 'Opponent', right_on = 'Franchise')
    print("abcd.shape : ",abcd.shape)
    
    prev_season = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//prev_season.csv")
    coaches = pd.read_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//"+team+"//coaches.csv",skiprows=1)
    e=clean_prev_season_and_coach(prev_season,coaches,team)
    
    abcde = pd.merge(left = abcd , right = e, how='left', left_on = 'Team_Name', right_on = 'Team_abv')
    print("abcde.shape : ",abcde.shape)
    
    return abcde

In [20]:
merge_df(team='ATL')

shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)


,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,B. Bogdanović · C. Capela · J. Collins · D. Hu...,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129,Dallas Mavericks,82,45,37,0.549,8398,8278,102.4,101.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225,Cleveland Cavaliers,236,127,109,0.538,24192,24058,102.5,101.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,B. Bogdanović · C. Capela · J. Collins · K. Hu...,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165,Detroit Pistons,438,213,225,0.486,44340,44530,101.2,101.7,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086,New Orleans Pelicans,41,21,20,0.512,3986,4019,97.2,98.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,B. Bogdanović · C. Capela · J. Collins · D. Hu...,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172,Washington Wizards,312,162,150,0.519,32954,32728,105.6,104.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [21]:
merge_df(team='LAL')

shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)


,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Tue, Oct 19, 2021",10:00p,LAL,NaN,Golden State Warriors,L,114,121,NaN,NaN,NaN,K. Bazemore · A. Davis · L. James · D. Jordan ...,45,95,.474,15,42,.357,9,19,.474,5,45,21,7,4,17,25,41,93,.441,14,39,.359,25,30,.833,9,50,30,9,2,17,18,101.1,107.3,112.8,.200,.442,.551,47.4,46.7,6.2,7.4,.553,14.1,10.9,.095,.516,13.8,81.6,.269,Golden State Warriors,430,258,172,0.600,46932,45758,109.1,106.4,Frank Vogel,2020,2022,3,225,127,98,.564,14.5,2,27,18,9,.667,1,1,LosAngelesLakers,LAL,0.583,3rd of 5,2.77,98.7,-0.5,109.9,-2.4,107.1,-5.2,Lost W. Conf. 1st Rnd.,M. Harrell (7.3)
1,1,2,"Fri, Oct 22, 2021",10:00p,LAL,NaN,Phoenix Suns,L,105,115,0,1,L 1,K. Bazemore · A. Davis · L. James · D. Jordan ...,34,86,.395,15,34,.441,22,29,.759,12,46,25,7,3,16,26,42,86,.488,10,32,.313,21,28,.750,7,47,31,8,2,11,22,104.5,114.5,100.5,.337,.395,.532,49.5,73.5,7.0,5.6,.483,13.9,23.1,.256,.547,10.1,82.9,.244,Phoenix Suns,259,146,113,0.564,28340,27809,109.4,107.4,Frank Vogel,2020,2022,3,225,127,98,.564,14.5,2,27,18,9,.667,1,1,LosAngelesLakers,LAL,0.583,3rd of 5,2.77,98.7,-0.5,109.9,-2.4,107.1,-5.2,Lost W. Conf. 1st Rnd.,M. Harrell (7.3)
2,2,3,"Sun, Oct 24, 2021",9:30p,LAL,NaN,Memphis Grizzlies,W,121,118,0,2,L 2,K. Bazemore · A. Davis · L. James · D. Jordan ...,45,84,.536,16,30,.533,15,22,.682,9,36,29,10,12,17,14,45,100,.450,13,35,.371,15,18,.833,18,49,25,13,4,17,20,120.4,117.4,100.5,.262,.357,.646,42.4,64.4,9.9,18.5,.631,15.4,22.5,.179,.515,13.6,60.0,.150,Memphis Grizzlies,99,61,38,0.616,10148,9747,102.5,98.5,Frank Vogel,2020,2022,3,225,127,98,.564,14.5,2,27,18,9,.667,1,1,LosAngelesLakers,LAL,0.583,3rd of 5,2.77,98.7,-0.5,109.9,-2.4,107.1,-5.2,Lost W. Conf. 1st Rnd.,M. Harrell (7.3)
3,3,4,"Tue, Oct 26, 2021",8:30p,LAL,@,San Antonio Spurs,W,125,121,1,2,W 1,K. Bazemore · A. Davis · D. Jordan · M. Monk ·...,50,105,.476,8,30,.267,17,22,.773,9,56,20,8,10,13,21,48,108,.444,13,36,.361,12,22,.545,16,59,33,10,6,15,22,107.7,104.3,105.1,.210,.286,.545,48.7,40.0,6.9,13.9,.514,10.2,17.3,.162,.505,11.3,74.6,.111,San Antonio Spurs,181,89,92,0.492,18915,19073,104.5,105.4,Frank Vogel,2020,2022,3,225,127,98,.564,14.5,2,27,18,9,.667,1,1,LosAngelesLakers,LAL,0.583,3rd of 5,2.77,98.7,-0.5,109.9,-2.4,107.1,-5.2,Lost W. Conf. 1st Rnd.,M. Harrell (7.3)
4,4,5,"Wed, Oct 27, 2021",8:00p,LAL,@,Oklahoma City Thunder,L,115,123,2,2,W 2,K. Bazemore · A. Davis · D. Jordan · M. Monk ·...,46,92,.500,11,35,.314,12,14,.857,10,40,29,8,2,17,22,43,86,.500,15,37,.405,22,29,.759,10,42,21,8,3,15,16,112.8,120.6,102.0,.152,.380,.586,48.8,63.0,7.8,4.1,.560,14.8,23.8,.130,.587,13.2,75.0,.256,Oklahoma City Thunder,260,151,109,0.581,27910,27630,107.3,106.3,Frank Vogel,2020,2022,3,225,127,98,.564,14.5,2,27,18,9,.667,1,1,LosAngelesLakers,LAL,0.583,3rd of 5,2.77,98.7,-0.5,109.9,-2.4,107.1,-5.2,Lost W. Conf. 1st Rnd.,M. Harrell (7.3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [22]:
merge_df(team='BRK')

shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)


,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Tue, Oct 19, 2021",7:30p,BRK,@,Milwaukee Bucks,L,104,127,NaN,NaN,NaN,N. Claxton · K. Durant · B. Griffin · J. Harde...,37,84,.440,17,32,.531,13,23,.565,5,44,19,3,9,12,17,48,105,.457,17,45,.378,14,18,.778,13,54,25,8,9,7,19,102.1,124.7,101.8,.274,.381,.552,44.9,51.4,2.9,15.0,.542,11.3,10.9,.155,.538,5.8,75.0,.133,Milwaukee Bucks,183,76,107,0.415,18551,19241,101.4,105.1,Steve Nash,2021,2022,2,154,92,62,.597,15.0,2,16,7,9,.438,0,0,BrooklynNets,BRK,0.667,2nd of 5,4.24,99.5,0.3,118.3,6.0,113.8,1.5,Lost E. Conf. Semis,K. Irving (7.4)
1,1,2,"Fri, Oct 22, 2021",7:30p,BRK,@,Philadelphia 76ers,W,114,109,0,1,L 1,N. Claxton · K. Durant · B. Griffin · J. Harde...,41,88,.466,12,38,.316,20,24,.833,11,47,26,6,6,12,16,43,91,.473,14,36,.389,9,11,.818,5,35,17,8,9,8,21,118.0,112.8,96.6,.273,.432,.578,57.3,63.4,6.2,10.9,.534,10.9,26.8,.227,.549,7.7,87.8,.099,Philadelphia 76ers,207,88,119,0.425,21155,21409,102.2,103.4,Steve Nash,2021,2022,2,154,92,62,.597,15.0,2,16,7,9,.438,0,0,BrooklynNets,BRK,0.667,2nd of 5,4.24,99.5,0.3,118.3,6.0,113.8,1.5,Lost E. Conf. Semis,K. Irving (7.4)
2,2,3,"Sun, Oct 24, 2021",4:00p,BRK,NaN,Charlotte Hornets,L,95,111,1,1,W 1,J. Carter · N. Claxton · K. Durant · J. Harden...,38,87,.437,9,35,.257,10,12,.833,9,44,18,10,8,17,19,43,94,.457,10,30,.333,15,19,.789,10,48,26,10,5,12,17,94.9,110.9,100.1,.138,.402,.515,47.8,47.4,10.0,12.5,.489,15.6,19.1,.115,.511,10.5,77.8,.160,Charlotte Hornets,118,62,56,0.525,11952,11898,101.3,100.8,Steve Nash,2021,2022,2,154,92,62,.597,15.0,2,16,7,9,.438,0,0,BrooklynNets,BRK,0.667,2nd of 5,4.24,99.5,0.3,118.3,6.0,113.8,1.5,Lost E. Conf. Semis,K. Irving (7.4)
3,3,4,"Mon, Oct 25, 2021",7:30p,BRK,NaN,Washington Wizards,W,104,90,1,2,L 1,B. Brown · K. Durant · B. Griffin · J. Harden ...,40,94,.426,10,35,.286,14,17,.824,7,54,23,6,4,9,19,33,95,.347,9,39,.231,15,20,.750,10,53,16,5,5,12,17,101.4,87.8,102.5,.181,.372,.512,50.5,57.5,5.9,7.1,.479,8.1,14.0,.149,.395,10.4,82.5,.158,Washington Wizards,199,92,107,0.462,20329,20493,102.2,103.0,Steve Nash,2021,2022,2,154,92,62,.597,15.0,2,16,7,9,.438,0,0,BrooklynNets,BRK,0.667,2nd of 5,4.24,99.5,0.3,118.3,6.0,113.8,1.5,Lost E. Conf. Semis,K. Irving (7.4)
4,4,5,"Wed, Oct 27, 2021",7:30p,BRK,NaN,Miami Heat,L,93,106,2,2,W 1,B. Brown · K. Durant · B. Griffin · J. Harden ...,33,85,.388,14,43,.326,13,14,.929,4,42,21,7,7,15,24,38,96,.396,8,29,.276,22,26,.846,17,62,21,6,2,14,19,92.0,104.8,101.1,.165,.506,.510,40.4,63.6,6.9,10.4,.471,14.1,8.2,.153,.438,11.5,69.1,.229,Miami Heat,133,55,78,0.414,12662,13034,95.2,98.0,Steve Nash,2021,2022,2,154,92,62,.597,15.0,2,16,7,9,.438,0,0,BrooklynNets,BRK,0.667,2nd of 5,4.24,99.5,0.3,118.3,6.0,113.8,1.5,Lost E. Conf. Semis,K. Irving (7.4)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [23]:
merge_df(team='NOP')

shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)


,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Wed, Oct 20, 2021",8:00p,NOP,NaN,Philadelphia 76ers,L,97,117,NaN,NaN,NaN,N. Alexander-Walker · D. Graham · J. Hart · B....,39,93,.419,13,30,.433,6,9,.667,10,44,28,5,6,13,16,44,85,.518,13,25,.520,16,20,.800,6,47,24,9,5,12,11,98.6,119.0,98.3,.097,.323,.500,48.4,71.8,5.1,10.0,.489,11.8,19.6,.065,.594,11.3,85.0,.188,Philadelphia 76ers,41,19,22,0.463,4004,3980,97.7,97.1,Willie Green,2022,2022,1,82,36,46,0.439,-5.0,1.0,6.0,2.0,4.0,0.333,NaN,NaN,NewOrleansPelicans,NOP,0.431,4th of 5,-0.2,100.1,0.9,113.5,1.2,113.8,1.5,NaN,Z. Williamson (8.7)
1,1,2,"Fri, Oct 22, 2021",8:00p,NOP,@,Chicago Bulls,L,112,128,0,1,L 1,N. Alexander-Walker · D. Graham · B. Ingram · ...,42,87,.483,15,29,.517,13,16,.813,9,42,26,7,5,16,20,47,89,.528,14,28,.500,20,25,.800,11,46,32,9,7,14,14,111.3,127.2,100.7,.184,.333,.595,47.7,61.9,7.0,8.2,.569,14.5,20.5,.149,.607,12.3,75.0,.225,Chicago Bulls,44,19,25,0.432,4412,4482,100.3,101.9,Willie Green,2022,2022,1,82,36,46,0.439,-5.0,1.0,6.0,2.0,4.0,0.333,NaN,NaN,NewOrleansPelicans,NOP,0.431,4th of 5,-0.2,100.1,0.9,113.5,1.2,113.8,1.5,NaN,Z. Williamson (8.7)
2,2,3,"Sat, Oct 23, 2021",8:00p,NOP,@,Minnesota Timberwolves,L,89,96,0,2,L 2,N. Alexander-Walker · D. Graham · B. Ingram · ...,31,89,.348,9,40,.225,18,23,.783,21,61,15,10,5,28,22,39,96,.406,12,38,.316,6,10,.600,13,47,22,13,8,19,25,86.2,93.0,103.3,.258,.449,.449,56.5,48.4,9.7,8.6,.399,22.0,38.2,.202,.469,15.9,75.5,.063,Minnesota Timberwolves,68,36,32,0.529,6989,6927,102.8,101.9,Willie Green,2022,2022,1,82,36,46,0.439,-5.0,1.0,6.0,2.0,4.0,0.333,NaN,NaN,NewOrleansPelicans,NOP,0.431,4th of 5,-0.2,100.1,0.9,113.5,1.2,113.8,1.5,NaN,Z. Williamson (8.7)
3,3,4,"Mon, Oct 25, 2021",8:00p,NOP,@,Minnesota Timberwolves,W,107,98,0,3,L 3,N. Alexander-Walker · D. Graham · B. Ingram · ...,39,87,.448,10,30,.333,19,23,.826,15,60,19,5,5,17,20,31,90,.344,15,50,.300,21,27,.778,11,41,20,10,7,11,19,109.5,100.3,97.7,.264,.345,.551,59.4,48.7,5.1,12.5,.506,14.9,33.3,.218,.428,9.7,80.4,.233,Minnesota Timberwolves,68,36,32,0.529,6989,6927,102.8,101.9,Willie Green,2022,2022,1,82,36,46,0.439,-5.0,1.0,6.0,2.0,4.0,0.333,NaN,NaN,NewOrleansPelicans,NOP,0.431,4th of 5,-0.2,100.1,0.9,113.5,1.2,113.8,1.5,NaN,Z. Williamson (8.7)
4,4,5,"Wed, Oct 27, 2021",7:30p,NOP,NaN,Atlanta Hawks,L,99,102,1,3,W 1,N. Alexander-Walker · D. Graham · B. Ingram · ...,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,107.0,110.2,92.5,.097,.387,.511,44.4,60.0,5.4,9.1,.489,8.5,20.9,.086,.458,9.6,62.5,.146,Atlanta Hawks,41,20,21,0.488,4019,3986,98.0,97.2,Willie Green,2022,2022,1,82,36,46,0.439,-5.0,1.0,6.0,2.0,4.0,0.333,NaN,NaN,NewOrleansPelicans,NOP,0.431,4th of 5,-0.2,100.1,0.9,113.5,1.2,113.8,1.5,NaN,Z. Williamson (8.7)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [24]:
merge_df(team='CHO')

shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)


,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Wed, Oct 20, 2021",7:00p,CHO,NaN,Indiana Pacers,W,123,122,NaN,NaN,NaN,L. Ball · M. Bridges · G. Hayward · K. Oubre ·...,46,107,.430,13,31,.419,18,27,.667,12,46,29,9,5,8,21,42,90,.467,17,47,.362,21,24,.875,8,51,29,2,10,16,24,113.2,112.2,108.7,.252,.290,.517,47.4,63.0,8.3,11.6,.491,6.3,21.8,.168,.561,13.7,81.0,.233,Indiana Pacers,121,51,70,0.421,12092,12464,99.9,103.0,James Borrego,2019,2022,4,301,138,163,0.458,-12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CharlotteHornets,CHO,0.458,4th of 5,-1.94,98.3,-0.9,110.9,-1.4,112.8,0.5,NaN,T. Rozier (5.2)
1,1,2,"Fri, Oct 22, 2021",7:00p,CHO,@,Cleveland Cavaliers,W,123,112,1,0,W 1,L. Ball · M. Bridges · G. Hayward · M. Plumlee...,48,91,.527,13,32,.406,14,17,.824,11,46,32,13,1,18,16,43,84,.512,9,31,.290,17,20,.850,7,36,26,9,3,18,15,119.5,108.8,102.9,.187,.352,.624,56.1,66.7,12.6,1.9,.599,15.5,27.5,.154,.565,16.2,83.3,.202,Cleveland Cavaliers,119,48,71,0.403,11787,12148,99.1,102.1,James Borrego,2019,2022,4,301,138,163,0.458,-12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CharlotteHornets,CHO,0.458,4th of 5,-1.94,98.3,-0.9,110.9,-1.4,112.8,0.5,NaN,T. Rozier (5.2)
2,2,3,"Sun, Oct 24, 2021",4:00p,CHO,@,Brooklyn Nets,W,111,95,2,0,W 2,L. Ball · M. Bridges · G. Hayward · K. Oubre ·...,43,94,.457,10,30,.333,15,19,.789,10,48,26,10,5,12,17,38,87,.437,9,35,.257,10,12,.833,9,44,18,10,8,17,19,110.9,94.9,100.1,.202,.319,.542,52.2,60.5,10.0,9.6,.511,10.5,22.2,.160,.489,15.6,80.9,.115,Brooklyn Nets,118,56,62,0.475,11898,11952,100.8,101.3,James Borrego,2019,2022,4,301,138,163,0.458,-12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CharlotteHornets,CHO,0.458,4th of 5,-1.94,98.3,-0.9,110.9,-1.4,112.8,0.5,NaN,T. Rozier (5.2)
3,3,4,"Mon, Oct 25, 2021",7:00p,CHO,NaN,Boston Celtics,L,129,140,3,0,W 3,L. Ball · M. Bridges · G. Hayward · K. Oubre ·...,46,103,.447,19,44,.432,18,28,.643,14,52,26,4,6,14,25,50,101,.495,19,50,.380,21,25,.840,11,53,33,8,9,12,23,115.3,125.2,101.3,.272,.427,.559,49.5,56.5,3.6,11.8,.539,10.8,25.0,.175,.589,9.7,77.6,.208,Boston Celtics,116,45,71,0.388,11646,12094,100.4,104.3,James Borrego,2019,2022,4,301,138,163,0.458,-12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CharlotteHornets,CHO,0.458,4th of 5,-1.94,98.3,-0.9,110.9,-1.4,112.8,0.5,NaN,T. Rozier (5.2)
4,4,5,"Wed, Oct 27, 2021",7:00p,CHO,@,Orlando Magic,W,120,111,3,1,L 1,L. Ball · M. Bridges · G. Hayward · K. Oubre ·...,43,83,.518,14,30,.467,20,24,.833,10,40,22,14,4,16,14,43,88,.489,15,40,.375,10,13,.769,8,33,22,6,2,18,21,121.6,112.4,98.7,.289,.361,.641,54.8,51.2,14.2,8.3,.602,14.6,28.6,.241,.574,16.1,78.9,.114,Orlando Magic,114,58,56,0.509,11606,11582,101.8,101.6,James Borrego,2019,2022,4,301,138,163,0.458,-12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CharlotteHornets,CHO,0.458,4th of 5,-1.94,98.3,-0.9,110.9,-1.4,112.8,0.5,NaN,T. Rozier (5.2)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [25]:
teams = ['ATL',
 'BOS',
 'BRK',
 'CHO',
 'CHI',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS']

In [26]:
len(teams)

30

In [27]:
all_team_list =[]

In [28]:
for team in teams:
    print(team)
    team_df = merge_df(team)
    print("Done with ",team)
    all_team_list.append(team_df)
    print()
    print()
    

ATL
shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)
Done with  ATL


BOS
shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)
Done with  BOS


BRK
shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)
Done with  BRK


CHO
shape of games_df (82, 13)
shape of line_ups_df (82, 2)
shape of gamelog_df (82, 33)
ab.shape :  (82, 46)
shape of game_advance_stats_df (82, 19)
abc.shape :  (82, 64)
abcd.shape :  (82, 73)
abcde.shape :  (82, 102)
Done with  CHO


CHI
shape of games_df (82, 13)
shape of line_ups_df 

In [29]:
all_team_df = pd.concat(all_team_list)

In [30]:
all_team_df.head()

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,B. Bogdanović · C. Capela · J. Collins · D. Hu...,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129,Dallas Mavericks,82,45,37,0.549,8398,8278,102.4,101.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225,Cleveland Cavaliers,236,127,109,0.538,24192,24058,102.5,101.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,B. Bogdanović · C. Capela · J. Collins · K. Hu...,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165,Detroit Pistons,438,213,225,0.486,44340,44530,101.2,101.7,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086,New Orleans Pelicans,41,21,20,0.512,3986,4019,97.2,98.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,B. Bogdanović · C. Capela · J. Collins · D. Hu...,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172,Washington Wizards,312,162,150,0.519,32954,32728,105.6,104.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)


In [31]:
all_team_df.Team.value_counts()

SacramentoKings          164
AtlantaHawks              82
BostonCeltics             82
UtahJazz                  82
TorontoRaptors            82
SanAntonioSpurs           82
PortlandTrailBlazers      82
PhoenixSuns               82
Philadelphiaers           82
OrlandoMagic              82
OklahomaCityThunder       82
NewYorkKnicks             82
NewOrleansPelicans        82
MinnesotaTimberwolves     82
MilwaukeeBucks            82
MiamiHeat                 82
MemphisGrizzlies          82
LosAngelesLakers          82
LosAngelesClippers        82
IndianaPacers             82
HoustonRockets            82
GoldenStateWarriors       82
DetroitPistons            82
DenverNuggets             82
DallasMavericks           82
ClevelandCavaliers        82
ChicagoBulls              82
CharlotteHornets          82
BrooklynNets              82
WashingtonWizards         82
Name: Team, dtype: int64

### sacramento is duplicated

In [98]:
all_team_df.Team.nunique()

30

In [32]:
all_team_df[all_team_df.Team=='SacramentoKings'].duplicated(subset=['G']).sum()

82

In [33]:
SAC_df = all_team_df[all_team_df.Team=='SacramentoKings']

In [34]:
SAC_df.shape

(164, 102)

In [35]:
SAC_df.drop_duplicates(subset=['G'],inplace=True)

In [36]:
SAC_df.shape

(82, 102)

In [37]:
all_team_df = all_team_df[all_team_df.Team!='SacramentoKings']

In [38]:
all_team_df.shape

(2378, 102)

In [39]:
all_team_df = pd.concat([all_team_df,SAC_df])

In [40]:
all_team_df.shape

(2460, 102)

In [44]:
all_team_df.head()

,idx,G,Date,Start (ET),Team_Name,Venue,Opponent,Result,Tm,Opp,W_Streak,L_Streak,Streak,Starting Lineup,Tm_FG,Tm_FGA,Tm_FG%,Tm_3P,Tm_3PA,Tm_3P%,Tm_FT,Tm_FTA,Tm_FT%,Tm_ORB,Tm_TRB,Tm_AST,Tm_STL,Tm_BLK,Tm_TOV,Tm_PF,Opp_FG,Opp_FGA,Opp_FG%,Opp_3P,Opp_3PA,Opp_3P%,Opp_FT,Opp_FTA,Opp_FT%,Opp_ORB,Opp_TRB,Opp_AST,Opp_STL,Opp_BLK,Opp_TOV,Opp_PF,Adv_Tm_ORtg,Adv_Tm_DRtg,Adv_Tm_Pace,Adv_Tm_FTr,Adv_Tm_3PAr,Adv_Tm_TS%,Adv_Tm_TRB%,Adv_Tm_AST%,Adv_Tm_STL%,Adv_Tm_BLK%,Adv_Tm_O_4F_eFG%,Adv_Tm_O_4F_TOV%,Adv_Tm_O_4F_ORB%,Adv_Tm_O_4F_FT/FGA,Adv_Tm_D_4F_eFG%,Adv_Tm_D_4F_TOV%,Adv_Tm_D_4F_DRB%,Adv_Tm_D_4F_FT/FGA,Franchise,H2H_G,H2H_W,H2H_L,H2H_W/L%,H2H_PS,H2H_PA,H2H_PS/G,H2H_PA/G,Coach,coach_From,coach_To,coach_Yrs,coach_G,coach_W,coach_L,coach_W/L%,coach_W > .500,coach_Plyfs,coach_G,coach_W,coach_L,coach_W/L%,coach_Conf,coach_Champ,Team,Team_abv,prev_W/L%,prev_Finish,prev_SRS,prev_Pace,prev_Rel_Pace,prev_ORtg,prev_Rel_ORtg,prev_DRtg,prev_Rel_DRtg,prev_Playoffs,prev_Top_WS
0,0,1,"Thu, Oct 21, 2021",7:30p,ATL,NaN,Dallas Mavericks,W,113,87,NaN,NaN,NaN,B. Bogdanović · C. Capela · J. Collins · D. Hu...,45,94,.479,15,35,.429,8,9,.889,6,55,31,8,9,13,16,31,93,.333,13,43,.302,12,13,.923,10,50,16,7,3,15,21,109.9,84.6,102.9,.096,.372,.577,52.4,68.9,7.8,18.0,.559,11.7,13.0,.085,.403,13.2,83.1,.129,Dallas Mavericks,82,45,37,0.549,8398,8278,102.4,101.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
1,1,2,"Sat, Oct 23, 2021",6:00p,ATL,@,Cleveland Cavaliers,L,95,101,1,0,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...,38,99,.384,10,34,.294,9,15,.600,17,54,20,5,3,9,23,37,89,.416,7,28,.250,20,27,.741,12,54,23,6,6,10,16,99.3,105.6,95.7,.152,.343,.450,50.0,52.6,5.2,4.9,.434,7.9,28.8,.091,.455,9.0,75.5,.225,Cleveland Cavaliers,236,127,109,0.538,24192,24058,102.5,101.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
2,2,3,"Mon, Oct 25, 2021",7:30p,ATL,NaN,Detroit Pistons,W,122,104,1,1,L 1,B. Bogdanović · C. Capela · J. Collins · K. Hu...,46,90,.511,12,32,.375,18,21,.857,10,49,24,11,3,13,19,40,91,.440,9,33,.273,15,18,.833,11,36,26,7,6,14,15,123.1,105.0,99.1,.233,.356,.615,57.6,52.2,11.1,5.2,.578,11.6,28.6,.200,.489,12.4,78.0,.165,Detroit Pistons,438,213,225,0.486,44340,44530,101.2,101.7,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
3,3,4,"Wed, Oct 27, 2021",7:30p,ATL,@,New Orleans Pelicans,W,102,99,2,1,W 1,B. Bogdanović · C. Capela · J. Collins · D. Hu...,40,96,.417,8,30,.267,14,17,.824,21,55,21,4,4,11,14,40,93,.430,11,36,.306,8,9,.889,9,44,24,5,6,9,19,110.2,107.0,92.5,.177,.313,.493,55.6,52.5,4.3,7.0,.458,9.6,37.5,.146,.489,8.5,79.1,.086,New Orleans Pelicans,41,21,20,0.512,3986,4019,97.2,98.0,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)
4,4,5,"Thu, Oct 28, 2021",7:00p,ATL,@,Washington Wizards,L,111,122,3,1,W 2,B. Bogdanović · C. Capela · J. Collins · D. Hu...,48,88,.545,6,21,.286,9,14,.643,6,43,26,4,3,13,16,46,99,.465,13,34,.382,17,21,.810,14,51,26,6,4,6,16,111.9,122.9,99.2,.159,.239,.589,45.7,54.2,4.0,4.6,.580,12.1,14.0,.102,.530,5.3,72.5,.172,Washington Wizards,312,162,150,0.519,32954,32728,105.6,104.9,Nate McMillan,2021,2022,2,120,70,50,.583,10.0,2,23,11,12,.478,0,0,AtlantaHawks,ATL,0.569,1st of 5,2.14,97.6,-1.6,115.7,3.4,113.3,1.0,Lost E. Conf. Finals,C. Capela (8.2)


In [41]:
all_team_df.Team.value_counts()

AtlantaHawks             82
BostonCeltics            82
WashingtonWizards        82
UtahJazz                 82
TorontoRaptors           82
SanAntonioSpurs          82
PortlandTrailBlazers     82
PhoenixSuns              82
Philadelphiaers          82
OrlandoMagic             82
OklahomaCityThunder      82
NewYorkKnicks            82
NewOrleansPelicans       82
MinnesotaTimberwolves    82
MilwaukeeBucks           82
MiamiHeat                82
MemphisGrizzlies         82
LosAngelesLakers         82
LosAngelesClippers       82
IndianaPacers            82
HoustonRockets           82
GoldenStateWarriors      82
DetroitPistons           82
DenverNuggets            82
DallasMavericks          82
ClevelandCavaliers       82
ChicagoBulls             82
CharlotteHornets         82
BrooklynNets             82
SacramentoKings          82
Name: Team, dtype: int64

In [42]:
all_team_df.to_csv("C://Users//user//Solo Dolo//NBA_Analysis//Game//All_team_with_lineups.csv",header=True,index=False)

In [43]:
all_team_df.isna().sum()[all_team_df.isna().sum()>1]

Venue            1230
W_Streak           30
L_Streak           30
Streak             30
coach_Plyfs       656
coach_G           656
coach_W           656
coach_L           656
coach_W/L%        656
coach_Conf       1066
coach_Champ      1066
prev_Playoffs    1148
dtype: int64

In [49]:
for gp ,gp_df in all_team_df.groupby('Team'):
    print(gp)
    print()
    print(gp_df['Starting Lineup'].nunique(),' different lineups')
    print((gp_df['Starting Lineup'].nunique()/gp_df.shape[0])*100)
    print()

AtlantaHawks

25  different lineups
30.48780487804878

BostonCeltics

21  different lineups
25.609756097560975

BrooklynNets

43  different lineups
52.4390243902439

CharlotteHornets

13  different lineups
15.853658536585366

ChicagoBulls

29  different lineups
35.36585365853659

ClevelandCavaliers

28  different lineups
34.146341463414636

DallasMavericks

22  different lineups
26.82926829268293

DenverNuggets

17  different lineups
20.73170731707317

DetroitPistons

22  different lineups
26.82926829268293

GoldenStateWarriors

30  different lineups
36.58536585365854

HoustonRockets

24  different lineups
29.268292682926827

IndianaPacers

37  different lineups
45.1219512195122

LosAngelesClippers

26  different lineups
31.70731707317073

LosAngelesLakers

41  different lineups
50.0

MemphisGrizzlies

17  different lineups
20.73170731707317

MiamiHeat

23  different lineups
28.04878048780488

MilwaukeeBucks

31  different lineups
37.80487804878049

MinnesotaTimberwolves

26  different